In [21]:
"""
This is the first part of TensorFlow tutorial
The idea is to show the basic computation patterns, 
and write simple models for hand-written digit classification on MNIST dataset
"""
import tensorflow as tf
import numpy as np

In [22]:
## Section 1: TF basics
#  1.1 Graphs and sessions
A = tf.Variable(2)
# del
B = tf.Variable(3)
C = A+B
print (C)
## TODO: Write a simple calculation C=A+B

Tensor("add_4:0", shape=(), dtype=int32)


In [23]:
## Build a session to execute the computation
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print (sess.run(C))

5


In [24]:
## close your previous session
## and clear its information
tf.reset_default_graph()
sess.close()

## Below is how you use an interactive session
## Can you see the difference?
sess = tf.InteractiveSession()
A = tf.Variable(2)
# del
B = tf.Variable(3)
C = A+B
tf.global_variables_initializer().run()
print (C.eval())

5


In [25]:
## close your previous session
## and clear its information
tf.reset_default_graph()
sess.close()

In [26]:
## Section 1: Building a model with TF
from tensorflow.examples.tutorials.mnist import input_data
file = "./MNIST"
mnist = input_data.read_data_sets(file, one_hot=True)

Extracting ./MNIST/train-images-idx3-ubyte.gz
Extracting ./MNIST/train-labels-idx1-ubyte.gz
Extracting ./MNIST/t10k-images-idx3-ubyte.gz
Extracting ./MNIST/t10k-labels-idx1-ubyte.gz


In [29]:
# Task 1
# Please read and run this example, 
# And see a minimal instance of TensorFlow
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
y = tf.nn.softmax(tf.matmul(x, W) + b)
cross_entropy = -tf.reduce_sum(y_*tf.log(y))
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
for i in range(300):
    batch = mnist.train.next_batch(100)
    train_step.run(feed_dict={x: batch[0], y_: batch[1]})

In [30]:
# Use the testing set to see the model accuracy
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_:mnist.test.labels}))
sess.close()
tf.reset_default_graph()

0.9006


In [9]:
# Task 2
# Now try by yourself to build a 2-conv + 1FC layer

sess = tf.InteractiveSession()
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))
sess.run(tf.global_variables_initializer())

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')

W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
x_image = tf.reshape(x, [-1,28,28,1])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)
# DEL from here
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1, W_fc2) + b_fc2
# ends del here
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



In [10]:
sess.run(tf.global_variables_initializer())
for i in range(300):
    batch = mnist.train.next_batch(100)
    if i%10 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))

    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

    #print("test accuracy %g"%accuracy.eval(feed_dict={
    #    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.07
step 10, training accuracy 0.22
step 20, training accuracy 0.37
step 30, training accuracy 0.57
step 40, training accuracy 0.72
step 50, training accuracy 0.82
step 60, training accuracy 0.82
step 70, training accuracy 0.84
step 80, training accuracy 0.87
step 90, training accuracy 0.88
step 100, training accuracy 0.87
step 110, training accuracy 0.92
step 120, training accuracy 0.91
step 130, training accuracy 0.9
step 140, training accuracy 0.94
step 150, training accuracy 0.84
step 160, training accuracy 0.91
step 170, training accuracy 0.91
step 180, training accuracy 0.92
step 190, training accuracy 0.9
step 200, training accuracy 0.92
step 210, training accuracy 0.95
step 220, training accuracy 0.87
step 230, training accuracy 0.93
step 240, training accuracy 0.92
step 250, training accuracy 0.91
step 260, training accuracy 0.95
step 270, training accuracy 0.93
step 280, training accuracy 0.93
step 290, training accuracy 0.99


In [11]:
print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))
sess.close()

test accuracy 0.9491


In [12]:
sess.close()

In [19]:
# Task 2.3
# Some "layer" API calls brought by TF
# This is not finished
sess = tf.InteractiveSession()
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])
keep_prob = tf.placeholder(tf.float32)
sess.run(tf.global_variables_initializer())

x_image = tf.reshape(x, [-1,28,28,1])
conv1=tf.layers.conv2d(inputs=x_image, kernel_size=[5,5], filters=32)
pool1=tf.layers.max_pooling2d(inputs=conv1, pool_size=2, strides=1)
# del here
conv2=tf.layers.conv2d(inputs=conv1, kernel_size=[5,5], filters=64)
pool2=tf.layers.max_pooling2d(inputs=conv2, pool_size=2, strides=1)
h_pool2_flat = tf.layers.flatten(pool2)
h_fc1_drop = tf.nn.dropout(h_pool2_flat, keep_prob)
h_fc2=tf.layers.dense(h_fc1_drop,1024,activation="relu")
y_conv=tf.layers.dense(h_pool2_flat,10)
# ends del here
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [20]:
sess.run(tf.global_variables_initializer())
for i in range(300):
    batch = mnist.train.next_batch(100)
    if i%10 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))

    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
    #print("test accuracy %g"%accuracy.eval(feed_dict={
    #    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))
print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))
sess.close()

step 0, training accuracy 0.17
step 10, training accuracy 0.51
step 20, training accuracy 0.71
step 30, training accuracy 0.75
step 40, training accuracy 0.88
step 50, training accuracy 0.87
step 60, training accuracy 0.84
step 70, training accuracy 0.87
step 80, training accuracy 0.9
step 90, training accuracy 0.89
step 100, training accuracy 0.88
step 110, training accuracy 0.93
step 120, training accuracy 0.92
step 130, training accuracy 0.91
step 140, training accuracy 0.87
step 150, training accuracy 0.91
step 160, training accuracy 0.87
step 170, training accuracy 0.96
step 180, training accuracy 0.89
step 190, training accuracy 0.9
step 200, training accuracy 0.88
step 210, training accuracy 0.92
step 220, training accuracy 0.93
step 230, training accuracy 0.91
step 240, training accuracy 0.9
step 250, training accuracy 0.98
step 260, training accuracy 0.96
step 270, training accuracy 0.92
step 280, training accuracy 0.95
step 290, training accuracy 0.93
test accuracy 0.9211
